In [ ]:
import numpy as np
import tensorflow as tf
import sklearn.preprocessing as prep
from tensorflow.examples.tutorials.mnist import input_data

In [12]:
import os
print(os.getcwd())
print("AutoEncoder")

/Users/zhangzhen
AutoEncoder


In [8]:
# Xavier  美 ['zeɪvɪr]

# define xavier initialization function, mean distribution or gauss distribution
# mean = 0, var = 2/(dim_in+dim_out) 


def xavier_init(dim_in, dim_out, constant=1):
    """
    dim_in: dimension of the input tensor 
    dim_out: dimension of the out tensor 
    """
    span = np.sqrt(6/(dim_in + dim_out))
    return tf.random_uniform((dim_in, dim_out), minval=-span, maxval=span, dtype=tf.float32)


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(xavier_init(1, 2)), "\n")
    print(sess.run(xavier_init(2, 2)), "\n")
    print(sess.run(xavier_init(5, 12)), "\n")

[[-1.1698401  -0.06624639]] 

[[ 0.7266238  -0.02556622]
 [ 0.8269533   0.94094336]] 

[[-0.42756602 -0.45622638  0.46571004 -0.5897876   0.26043653  0.26666903
   0.08165872  0.20378679  0.3700722  -0.05493122  0.35615778 -0.24716526]
 [ 0.03058684  0.14769298  0.47950304 -0.5812248   0.30964786 -0.2709414
   0.02204543  0.3706919   0.42468882 -0.26162592  0.10810822 -0.12229562]
 [ 0.28569776  0.24804258  0.16488826  0.14234954  0.35423273  0.26771605
  -0.05920851 -0.441086   -0.20098042 -0.07493913 -0.10899445  0.18068445]
 [-0.56379193 -0.19462979  0.496516    0.09247845 -0.5843429  -0.2950208
   0.26738435  0.16739517 -0.50270087 -0.15485126  0.25061792  0.09368908]
 [-0.3591723   0.16769278 -0.49909198 -0.26201543  0.5462408  -0.5799796
   0.5638156   0.05251139  0.43511105  0.24451089 -0.33938935 -0.10266507]] 



In [13]:
class AdditiveGaussianNoiseAutoEncoder:
    
    def __init__(self, n_input, n_hidden, transfer_function=tf.nn.softplus, optimizer=tf.train.AdamOptimizer(), scale=0.1):
        self.n_input = n_input
        self.n_hidden = n_hidden
        self.transfer = transfer_function
        self.scale = tf.placeholder(tf.float32)
        self.training_scale = scale

        network_weights = self._initialize_weights()
        self.weights = network_weights
        
        with tf.name_scope("input"):
            self.x = tf.placeholder(tf.float32, [None, self.n_input])
        
        with tf.name_scope("hidden"):
            self.hidden = self.transfer(
                tf.add(tf.matmul(self.x + scale*tf.random_normal((n_input,)), self.weights['w1']),
                       self.weights['b1'])
            )
        with tf.name_scope("output"):
            self.reconstruction = tf.add(tf.matmul(self.hidden, self.weights['w2']), self.weights['b2'])
    
        with tf.name_scope('loss'):
            self.cost = 0.5 * tf.reduce_mean(tf.pow(tf.subtract(self.reconstruction, self.x), 2.0))
        
        self.optimizer = optimizer.minimize(self.cost)
        
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
    
    def _initialize_weights(self):
        all_weights = dict()
        all_weights['w1'] = tf.Variable(xavier_init(self.n_input, self.n_hidden))
        all_weights['b1'] = tf.Variable(tf.zeros([self.n_hidden], dtype=tf.float32))
        
        all_weights['w2'] = tf.Variable(tf.zeros([self.n_hidden, self.n_input], dtype=tf.float32))
        all_weights['b2'] = tf.Variable(tf.zeros([self.n_input], dtype=tf.float32))
        return all_weights

    def partial_fit(self, X):
        cost, opt = self.sess.run((self.cost, self.optimizer),feed_dict={self.x: X, self.scale: self.training_scale})
        return cost
    
    # loss function
    def calc_total_cost(self, X):
        return self.sess.run(self.cost, feed_dict={self.x: X, self.scale: self.training_scale})


mnist = input_data.read_data_sets('/Users/zhangzhen/gitRepository/AnalyticsVidhya/_tensorflow/MNIST_data', one_hot=True)
with tf.Graph().as_default():

    def standard_scale(X_train, X_test):
            preprocessor = prep.StandardScaler().fit(X_train)
            X_train = preprocessor.transform(X_train)
            X_test = preprocessor.transform(X_test)
            return X_train, X_test
    
    def get_random_block_from_data(data, batch_size):
            start_index = np.random.randint(0, len(data)-batch_size)
            return data[start_index:(start_index+batch_size)]

    X_train, X_test = standard_scale(mnist.train.images, mnist.test.images)
    
    n_samples = int(mnist.train.num_examples)        
    training_epochs = 20
    batch_size = 128
    display_step = 2
    
    autoencoder = AdditiveGaussianNoiseAutoEncoder(n_input=784,
                                                   n_hidden=200,
                                                   optimizer=tf.train.AdamOptimizer(learning_rate=0.002),
                                                   scale=0.01)
    
    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(n_samples/batch_size)
        for i in range(total_batch):
            batch_xs = get_random_block_from_data(X_train, batch_size)
            # cost = autoencoder.partial_fit(batch_xs)
            cost = autoencoder.partial_fit(batch_xs)
            avg_cost += cost / n_samples * batch_size 
        
        if epoch % display_step == 0:
            print('Epoch:', '%04d' % (epoch+1), 'cost=', '{:.9f}'.format(avg_cost))
    print('Total cost:'+str(autoencoder.calc_total_cost(X_test)))

Extracting /Users/zhangzhen/gitRepository/AnalyticsVidhya/_tensorflow/MNIST_data/train-images-idx3-ubyte.gz


Extracting /Users/zhangzhen/gitRepository/AnalyticsVidhya/_tensorflow/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /Users/zhangzhen/gitRepository/AnalyticsVidhya/_tensorflow/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /Users/zhangzhen/gitRepository/AnalyticsVidhya/_tensorflow/MNIST_data/t10k-labels-idx1-ubyte.gz


Epoch: 0001 cost= 0.167803666


Epoch: 0003 cost= 0.112595153


Epoch: 0005 cost= 0.101961227


...

Epoch: 0013 cost= 0.102885709


Epoch: 0015 cost= 0.096853506


Epoch: 0017 cost= 0.097080449


Epoch: 0019 cost= 0.089853504


Total cost:0.09154988
